In [2]:
import sys
!{sys.executable} -m pip install --upgrade google-cloud-bigquery google-cloud-bigquery-storage pandas-gbq

  Attempting uninstall: google-cloud-bigquery-storage
    Found existing installation: google-cloud-bigquery-storage 2.33.1
    Uninstalling google-cloud-bigquery-storage-2.33.1:
      Successfully uninstalled google-cloud-bigquery-storage-2.33.1


In [4]:
from google.cloud import bigquery

key_path = r"ngoc716-15ba76a1039c.json"


client = bigquery.Client.from_service_account_json(key_path)

In [6]:
query = """
SELECT *
FROM `bigquery-public-data.google_analytics_sample.ga_sessions_*`
lIMIT 10
"""
df = client.query(query).to_dataframe()

In [8]:
# 2️⃣ Chỉ định bảng muốn xem (ví dụ: ngày 20170801)
table_id = "bigquery-public-data.google_analytics_sample.ga_sessions"

# 3️⃣ Lấy schema của bảng
table = client.get_table(table_id)

# 4️⃣ Hàm đệ quy in cấu trúc trường (vì có nhiều field lồng nhau)
def print_schema(fields, indent=0):
    for field in fields:
        print(" " * indent + f"- {field.name} ({field.field_type})")
        if field.fields:  # Nếu là RECORD (STRUCT)
            print_schema(field.fields, indent + 4)

# 5️⃣ In danh sách các trường
print(f"📋 Schema của bảng: {table_id}\n")
print_schema(table.schema)

📋 Schema của bảng: bigquery-public-data.google_analytics_sample.ga_sessions_*

- visitorId (INTEGER)
- visitNumber (INTEGER)
- visitId (INTEGER)
- visitStartTime (INTEGER)
- date (STRING)
- totals (RECORD)
    - visits (INTEGER)
    - hits (INTEGER)
    - pageviews (INTEGER)
    - timeOnSite (INTEGER)
    - bounces (INTEGER)
    - transactions (INTEGER)
    - transactionRevenue (INTEGER)
    - newVisits (INTEGER)
    - screenviews (INTEGER)
    - uniqueScreenviews (INTEGER)
    - timeOnScreen (INTEGER)
    - totalTransactionRevenue (INTEGER)
    - sessionQualityDim (INTEGER)
- trafficSource (RECORD)
    - referralPath (STRING)
    - campaign (STRING)
    - source (STRING)
    - medium (STRING)
    - keyword (STRING)
    - adContent (STRING)
    - adwordsClickInfo (RECORD)
        - campaignId (INTEGER)
        - adGroupId (INTEGER)
        - creativeId (INTEGER)
        - criteriaId (INTEGER)
        - page (INTEGER)
        - slot (STRING)
        - criteriaParameters (STRING)
       

In [12]:
from google.cloud import bigquery
import pandas as pd

# 2️⃣ Viết truy vấn chọn chỉ các trường cần thiết
query = """
SELECT
  -- 🧩 Thông tin người dùng & phiên
  fullVisitorId,
  visitId,
  visitNumber,
  date,

  -- 💻 Thiết bị
  device.deviceCategory AS deviceCategory,
  device.browser AS browser,
  device.operatingSystem AS operatingSystem,

  -- 🌍 Địa lý
  geoNetwork.country AS country,
  geoNetwork.city AS city,

  -- 🚦 Nguồn traffic
  trafficSource.source AS source,
  trafficSource.medium AS medium,
  trafficSource.campaign AS campaign,
  trafficSource.isTrueDirect AS isTrueDirect,

  -- 📈 Chỉ số hành vi
  totals.pageviews AS pageviews,
  totals.hits AS hits,
  totals.timeOnSite AS time_on_site,
  totals.bounces AS bounces,
  totals.newVisits AS newVisits,
  totals.transactions AS transactions,
  totals.transactionRevenue AS transactionRevenue,
  totals.sessionQualityDim AS sessionQualityDim

FROM
  `bigquery-public-data.google_analytics_sample.ga_sessions_20170801`
WHERE
  totals.pageviews IS NOT NULL
"""

# 3️⃣ Thực thi truy vấn và đưa về DataFrame
df = client.query(query).to_dataframe()

# 4️⃣ (Tuỳ chọn) Loại bỏ các dòng toàn null hoặc duplicate
df = df.drop_duplicates()
df = df.dropna(subset=["fullVisitorId", "pageviews"])

# 5️⃣ Xem kết quả
print("✅ Dữ liệu sau khi chọn trường cần thiết:")
print(df.info())
print(df.head())

✅ Dữ liệu sau khi chọn trường cần thiết:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2556 entries, 0 to 2555
Data columns (total 21 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   fullVisitorId       2556 non-null   object 
 1   visitId             2556 non-null   Int64  
 2   visitNumber         2556 non-null   Int64  
 3   date                2556 non-null   object 
 4   deviceCategory      2556 non-null   object 
 5   browser             2556 non-null   object 
 6   operatingSystem     2556 non-null   object 
 7   country             2556 non-null   object 
 8   city                2556 non-null   object 
 9   source              2556 non-null   object 
 10  medium              2556 non-null   object 
 11  campaign            2556 non-null   object 
 12  isTrueDirect        872 non-null    boolean
 13  pageviews           2556 non-null   Int64  
 14  hits                2556 non-null   Int64  
 15  time_on_site  

In [1]:
import pandas as pd

# Đọc file
df = pd.read_csv("GA_CustomerBehavior.csv")

# Xem kích thước và thông tin
print("✅ Dữ liệu gốc:", df.shape)
print("\n📋 Thông tin các cột:")
print(df.info())

# Xem vài dòng đầu
print("\n🔍 5 dòng đầu:")
df.head()


✅ Dữ liệu gốc: (2556, 21)

📋 Thông tin các cột:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2556 entries, 0 to 2555
Data columns (total 21 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   fullVisitorId       2556 non-null   uint64 
 1   visitId             2556 non-null   int64  
 2   visitNumber         2556 non-null   int64  
 3   date                2556 non-null   int64  
 4   deviceCategory      2556 non-null   object 
 5   browser             2556 non-null   object 
 6   operatingSystem     2556 non-null   object 
 7   country             2556 non-null   object 
 8   city                2556 non-null   object 
 9   source              2556 non-null   object 
 10  medium              2556 non-null   object 
 11  campaign            2556 non-null   object 
 12  isTrueDirect        872 non-null    object 
 13  pageviews           2556 non-null   int64  
 14  hits                2556 non-null   int64  
 15  time_on

,fullVisitorId,visitId,visitNumber,date,deviceCategory,browser,operatingSystem,country,city,source,...,campaign,isTrueDirect,pageviews,hits,time_on_site,bounces,newVisits,transactions,transactionRevenue,sessionQualityDim
0,3418334011779872055,1501591568,1,20170801,desktop,Chrome,Windows,Greece,not available in demo dataset,(direct),...,(not set),NaN,1,1,NaN,1.0,1.0,NaN,NaN,1
1,2474397855041322408,1501589647,2,20170801,desktop,Chrome,Windows,India,Mumbai,analytics.google.com,...,(not set),NaN,1,1,NaN,1.0,NaN,NaN,NaN,1
2,5870462820713110108,1501616621,1,20170801,desktop,Chrome,Windows,United Kingdom,not available in demo dataset,analytics.google.com,...,(not set),NaN,1,1,NaN,1.0,1.0,NaN,NaN,1
3,9397809171349480379,1501601200,1,20170801,desktop,Firefox,Windows,United States,Dallas,analytics.google.com,...,(not set),NaN,1,1,NaN,1.0,1.0,NaN,NaN,1
4,6089902943184578335,1501615525,1,20170801,desktop,Chrome,Windows,United States,not available in demo dataset,adwords.google.com,...,(not set),NaN,1,1,NaN,1.0,1.0,NaN,NaN,1


In [2]:
# 🧮 Các cột định lượng (số)
num_cols = [
    "pageviews", "hits", "time_on_site", "bounces",
    "newVisits", "transactions", "transactionRevenue", "sessionQualityDim"
]

# 🏷️ Các cột định tính (phân loại)
cat_cols = [
    "deviceCategory", "browser", "operatingSystem",
    "country", "city", "source", "medium", "campaign"
]

print("🧮 Cột định lượng:", num_cols)
print("🏷️ Cột định tính:", cat_cols)


🧮 Cột định lượng: ['pageviews', 'hits', 'time_on_site', 'bounces', 'newVisits', 'transactions', 'transactionRevenue', 'sessionQualityDim']
🏷️ Cột định tính: ['deviceCategory', 'browser', 'operatingSystem', 'country', 'city', 'source', 'medium', 'campaign']


### Định lượng

In [6]:
df_num = df[num_cols] 
print("Bảng dữ liệu định lượng:")
df_num.head()

Bảng dữ liệu định lượng:


,pageviews,hits,time_on_site,bounces,newVisits,transactions,transactionRevenue,sessionQualityDim
0,1,1,NaN,1.0,1.0,NaN,NaN,1
1,1,1,NaN,1.0,NaN,NaN,NaN,1
2,1,1,NaN,1.0,1.0,NaN,NaN,1
3,1,1,NaN,1.0,1.0,NaN,NaN,1
4,1,1,NaN,1.0,1.0,NaN,NaN,1


In [8]:
df_num = df[num_cols].copy()
print("Trước xử lý:")
df_num.head()

Trước xử lý:


,pageviews,hits,time_on_site,bounces,newVisits,transactions,transactionRevenue,sessionQualityDim
0,1,1,NaN,1.0,1.0,NaN,NaN,1
1,1,1,NaN,1.0,NaN,NaN,NaN,1
2,1,1,NaN,1.0,1.0,NaN,NaN,1
3,1,1,NaN,1.0,1.0,NaN,NaN,1
4,1,1,NaN,1.0,1.0,NaN,NaN,1


In [10]:
print("\nSố lượng giá trị thiếu mỗi cột:")
print(df_num.isna().sum())


Số lượng giá trị thiếu mỗi cột:
pageviews                0
hits                     0
time_on_site          1246
bounces               1318
newVisits              684
transactions          2513
transactionRevenue    2513
sessionQualityDim        0
dtype: int64


In [12]:
# Xử lý dữ liệu bị thiếu
from sklearn.impute import SimpleImputer
import numpy as np

# Cột dạng tỷ lệ/đếm => điền 0
cols_zero = ["bounces", "newVisits", "transactions", "transactionRevenue"]
df_num[cols_zero] = df_num[cols_zero].fillna(0)

# Các cột còn lại => điền median
cols_median = [col for col in df_num.columns if col not in cols_zero]
imputer = SimpleImputer(strategy='median')
df_num[cols_median] = imputer.fit_transform(df_num[cols_median])


In [13]:
df_num.head()

,pageviews,hits,time_on_site,bounces,newVisits,transactions,transactionRevenue,sessionQualityDim
0,1.0,1.0,96.5,1.0,1.0,0.0,0.0,1.0
1,1.0,1.0,96.5,1.0,0.0,0.0,0.0,1.0
2,1.0,1.0,96.5,1.0,1.0,0.0,0.0,1.0
3,1.0,1.0,96.5,1.0,1.0,0.0,0.0,1.0
4,1.0,1.0,96.5,1.0,1.0,0.0,0.0,1.0


In [16]:
print("✅ Đã xử lý missing values.")
print(df_num.isna().sum())


✅ Đã xử lý missing values.
pageviews             0
hits                  0
time_on_site          0
bounces               0
newVisits             0
transactions          0
transactionRevenue    0
sessionQualityDim     0
dtype: int64


In [18]:
# Loại bỏ giá trị ngoại lai
import pandas as pd

# ---Tính IQR cho từng cột ---
Q1 = df_num.quantile(0.25)
Q3 = df_num.quantile(0.75)
IQR = Q3 - Q1

# ---Xác định ngưỡng dưới & ngưỡng trên ---
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# --- Giới hạn (clip) các giá trị vượt ngoài khoảng ---
df_num_clipped = df_num.clip(lower=lower_bound, upper=upper_bound, axis=1)

print("Đã xử lý ngoại lai bằng phương pháp IQR (cắt ngưỡng).")


Đã xử lý ngoại lai bằng phương pháp IQR (cắt ngưỡng).


In [20]:
print(" Kiểm tra lại phân phối sau khi xử lý:")
print(df_num_clipped.describe())

 Kiểm tra lại phân phối sau khi xử lý:
         pageviews         hits  time_on_site      bounces    newVisits  \
count  2556.000000  2556.000000   2556.000000  2556.000000  2556.000000   
mean      2.953443     3.415102     96.220559     0.484351     0.732394   
std       2.649629     3.438847     11.506164     0.499853     0.442797   
min       1.000000     1.000000     79.375000     0.000000     0.000000   
25%       1.000000     1.000000     91.750000     0.000000     0.000000   
50%       2.000000     2.000000     96.500000     0.000000     1.000000   
75%       4.000000     5.000000    100.000000     1.000000     1.000000   
max       8.500000    11.000000    112.375000     1.000000     1.000000   

       transactions  transactionRevenue  sessionQualityDim  
count        2556.0              2556.0             2556.0  
mean            0.0                 0.0                1.0  
std             0.0                 0.0                0.0  
min             0.0                 0.0  

In [22]:
df_num_clipped.head()

,pageviews,hits,time_on_site,bounces,newVisits,transactions,transactionRevenue,sessionQualityDim
0,1.0,1.0,96.5,1.0,1.0,0.0,0.0,1.0
1,1.0,1.0,96.5,1.0,0.0,0.0,0.0,1.0
2,1.0,1.0,96.5,1.0,1.0,0.0,0.0,1.0
3,1.0,1.0,96.5,1.0,1.0,0.0,0.0,1.0
4,1.0,1.0,96.5,1.0,1.0,0.0,0.0,1.0


In [24]:
# Chuẩn hóa dữ liệu - scaling
from sklearn.preprocessing import StandardScaler

# 🔹 Khởi tạo scaler
scaler = StandardScaler()

# 🔹 Scale các cột này (chuẩn hóa z-score)
df_scaled = df_num.copy()
df_scaled[num_cols] = scaler.fit_transform(df_scaled[num_cols])

print(" Đã scale xong các trường định lượng (chuẩn hóa Z-score).")
df_scaled.head()



 Đã scale xong các trường định lượng (chuẩn hóa Z-score).


,pageviews,hits,time_on_site,bounces,newVisits,transactions,transactionRevenue,sessionQualityDim
0,-0.411779,-0.353582,-0.266785,1.031804,0.604471,-0.128196,-0.049082,-0.293608
1,-0.411779,-0.353582,-0.266785,1.031804,-1.654340,-0.128196,-0.049082,-0.293608
2,-0.411779,-0.353582,-0.266785,1.031804,0.604471,-0.128196,-0.049082,-0.293608
3,-0.411779,-0.353582,-0.266785,1.031804,0.604471,-0.128196,-0.049082,-0.293608
4,-0.411779,-0.353582,-0.266785,1.031804,0.604471,-0.128196,-0.049082,-0.293608


In [26]:
print("\nThống kê sau chuẩn hóa:")
print(df_scaled[num_cols].describe().T)


Thống kê sau chuẩn hóa:
                     count          mean       std       min       25%  \
pageviews           2556.0  0.000000e+00  1.000196 -0.411779 -0.411779   
hits                2556.0 -4.447842e-17  1.000196 -0.353582 -0.353582   
time_on_site        2556.0 -4.447842e-17  1.000196 -0.484289 -0.277603   
bounces             2556.0  0.000000e+00  1.000196 -0.969176 -0.969176   
newVisits           2556.0 -2.223921e-17  1.000196 -1.654340 -1.654340   
transactions        2556.0 -2.223921e-17  1.000196 -0.128196 -0.128196   
transactionRevenue  2556.0 -1.111960e-17  1.000196 -0.049082 -0.049082   
sessionQualityDim   2556.0 -2.223921e-17  1.000196 -0.293608 -0.293608   

                         50%       75%        max  
pageviews          -0.286227 -0.035121  18.923333  
hits               -0.268937 -0.015002  25.124583  
time_on_site       -0.266785 -0.258813  11.937400  
bounces            -0.969176  1.031804   1.031804  
newVisits           0.604471  0.604471   0.60447

In [28]:
df_scaled.head()

,pageviews,hits,time_on_site,bounces,newVisits,transactions,transactionRevenue,sessionQualityDim
0,-0.411779,-0.353582,-0.266785,1.031804,0.604471,-0.128196,-0.049082,-0.293608
1,-0.411779,-0.353582,-0.266785,1.031804,-1.654340,-0.128196,-0.049082,-0.293608
2,-0.411779,-0.353582,-0.266785,1.031804,0.604471,-0.128196,-0.049082,-0.293608
3,-0.411779,-0.353582,-0.266785,1.031804,0.604471,-0.128196,-0.049082,-0.293608
4,-0.411779,-0.353582,-0.266785,1.031804,0.604471,-0.128196,-0.049082,-0.293608


### Định tính

In [32]:
df_cat = df[cat_cols]   
print("Bảng dữ liệu định tính:")
df_cat.head()

Bảng dữ liệu định tính:


,deviceCategory,browser,operatingSystem,country,city,source,medium,campaign
0,desktop,Chrome,Windows,Greece,not available in demo dataset,(direct),(none),(not set)
1,desktop,Chrome,Windows,India,Mumbai,analytics.google.com,referral,(not set)
2,desktop,Chrome,Windows,United Kingdom,not available in demo dataset,analytics.google.com,referral,(not set)
3,desktop,Firefox,Windows,United States,Dallas,analytics.google.com,referral,(not set)
4,desktop,Chrome,Windows,United States,not available in demo dataset,adwords.google.com,referral,(not set)


In [34]:
# Làm sạch dữ liệu định tính
# Làm sạch giá trị text
df_cat_clean = df_cat.copy()

# Chuẩn hóa chuỗi
for col in df_cat_clean.columns:
    df_cat_clean[col] = df_cat_clean[col].astype(str).str.lower().str.strip()

# Thay các giá trị không hợp lệ bằng 'unknown'
invalid_vals = ["(not set)", "not available in demo dataset", "(none)", "nan"]
df_cat_clean.replace(invalid_vals, "unknown", inplace=True)

print(" Đã làm sạch dữ liệu định tính:")
df_cat_clean.head()


 Đã làm sạch dữ liệu định tính:


,deviceCategory,browser,operatingSystem,country,city,source,medium,campaign
0,desktop,chrome,windows,greece,unknown,(direct),unknown,unknown
1,desktop,chrome,windows,india,mumbai,analytics.google.com,referral,unknown
2,desktop,chrome,windows,united kingdom,unknown,analytics.google.com,referral,unknown
3,desktop,firefox,windows,united states,dallas,analytics.google.com,referral,unknown
4,desktop,chrome,windows,united states,unknown,adwords.google.com,referral,unknown


In [36]:
# Mã hóa bằng LabelEncoder (phù hợp cho phân cụm)
from sklearn.preprocessing import LabelEncoder
df_cat_label = df_cat_clean.copy()
for col in df_cat_label.columns:
    le = LabelEncoder()
    df_cat_label[col] = le.fit_transform(df_cat_label[col])

print("✅ Dữ liệu định tính sau khi Label Encoding (dùng cho phân cụm):")
display(df_cat_label.head())

✅ Dữ liệu định tính sau khi Label Encoding (dùng cho phân cụm):


,deviceCategory,browser,operatingSystem,country,city,source,medium,campaign
0,0,2,7,30,52,0,4,1
1,0,2,7,36,32,2,3,1
2,0,2,7,89,52,2,3,1
3,0,5,7,90,13,2,3,1
4,0,2,7,90,52,1,3,1


In [38]:
df_cat_label.head()

,deviceCategory,browser,operatingSystem,country,city,source,medium,campaign
0,0,2,7,30,52,0,4,1
1,0,2,7,36,32,2,3,1
2,0,2,7,89,52,2,3,1
3,0,5,7,90,13,2,3,1
4,0,2,7,90,52,1,3,1


In [40]:
df_clean = pd.concat([
    df[["fullVisitorId", "visitId", "visitNumber", "date"]].reset_index(drop=True),
    df_scaled.reset_index(drop=True),
    df_cat_label.reset_index(drop=True)
], axis=1)


In [42]:
df_clean = pd.concat([
    df[["fullVisitorId", "visitId", "visitNumber", "date"]].astype({"fullVisitorId": "string"}).reset_index(drop=True),
    df_scaled.reset_index(drop=True),
    df_cat_label.reset_index(drop=True)
], axis=1)

In [44]:
df_clean.head()

,fullVisitorId,visitId,visitNumber,date,pageviews,hits,time_on_site,bounces,newVisits,transactions,transactionRevenue,sessionQualityDim,deviceCategory,browser,operatingSystem,country,city,source,medium,campaign
0,3418334011779872055,1501591568,1,20170801,-0.411779,-0.353582,-0.266785,1.031804,0.604471,-0.128196,-0.049082,-0.293608,0,2,7,30,52,0,4,1
1,2474397855041322408,1501589647,2,20170801,-0.411779,-0.353582,-0.266785,1.031804,-1.654340,-0.128196,-0.049082,-0.293608,0,2,7,36,32,2,3,1
2,5870462820713110108,1501616621,1,20170801,-0.411779,-0.353582,-0.266785,1.031804,0.604471,-0.128196,-0.049082,-0.293608,0,2,7,89,52,2,3,1
3,9397809171349480379,1501601200,1,20170801,-0.411779,-0.353582,-0.266785,1.031804,0.604471,-0.128196,-0.049082,-0.293608,0,5,7,90,13,2,3,1
4,6089902943184578335,1501615525,1,20170801,-0.411779,-0.353582,-0.266785,1.031804,0.604471,-0.128196,-0.049082,-0.293608,0,2,7,90,52,1,3,1


In [46]:
# 💾 Lưu dữ liệu sau xử lý
output_path = "ga_session.csv"
df_clean.to_csv(output_path, index=False, encoding='utf-8-sig')

print("✅ File đã lưu thành công:", output_path)


✅ File đã lưu thành công: ga_session.csv


In [48]:
import pandas as pd
df_clean = pd.read_csv("ga_session.csv")
df_clean.head()

,fullVisitorId,visitId,visitNumber,date,pageviews,hits,time_on_site,bounces,newVisits,transactions,transactionRevenue,sessionQualityDim,deviceCategory,browser,operatingSystem,country,city,source,medium,campaign
0,3418334011779872055,1501591568,1,20170801,-0.411779,-0.353582,-0.266785,1.031804,0.604471,-0.128196,-0.049082,-0.293608,0,2,7,30,52,0,4,1
1,2474397855041322408,1501589647,2,20170801,-0.411779,-0.353582,-0.266785,1.031804,-1.654340,-0.128196,-0.049082,-0.293608,0,2,7,36,32,2,3,1
2,5870462820713110108,1501616621,1,20170801,-0.411779,-0.353582,-0.266785,1.031804,0.604471,-0.128196,-0.049082,-0.293608,0,2,7,89,52,2,3,1
3,9397809171349480379,1501601200,1,20170801,-0.411779,-0.353582,-0.266785,1.031804,0.604471,-0.128196,-0.049082,-0.293608,0,5,7,90,13,2,3,1
4,6089902943184578335,1501615525,1,20170801,-0.411779,-0.353582,-0.266785,1.031804,0.604471,-0.128196,-0.049082,-0.293608,0,2,7,90,52,1,3,1


In [50]:
import pandas as pd
import numpy as np

df = pd.read_csv("ga_session.csv")

print(df.info())
df.head()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2556 entries, 0 to 2555
Data columns (total 20 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   fullVisitorId       2556 non-null   uint64 
 1   visitId             2556 non-null   int64  
 2   visitNumber         2556 non-null   int64  
 3   date                2556 non-null   int64  
 4   pageviews           2556 non-null   float64
 5   hits                2556 non-null   float64
 6   time_on_site        2556 non-null   float64
 7   bounces             2556 non-null   float64
 8   newVisits           2556 non-null   float64
 9   transactions        2556 non-null   float64
 10  transactionRevenue  2556 non-null   float64
 11  sessionQualityDim   2556 non-null   float64
 12  deviceCategory      2556 non-null   int64  
 13  browser             2556 non-null   int64  
 14  operatingSystem     2556 non-null   int64  
 15  country             2556 non-null   int64  
 16  city  

,fullVisitorId,visitId,visitNumber,date,pageviews,hits,time_on_site,bounces,newVisits,transactions,transactionRevenue,sessionQualityDim,deviceCategory,browser,operatingSystem,country,city,source,medium,campaign
0,3418334011779872055,1501591568,1,20170801,-0.411779,-0.353582,-0.266785,1.031804,0.604471,-0.128196,-0.049082,-0.293608,0,2,7,30,52,0,4,1
1,2474397855041322408,1501589647,2,20170801,-0.411779,-0.353582,-0.266785,1.031804,-1.654340,-0.128196,-0.049082,-0.293608,0,2,7,36,32,2,3,1
2,5870462820713110108,1501616621,1,20170801,-0.411779,-0.353582,-0.266785,1.031804,0.604471,-0.128196,-0.049082,-0.293608,0,2,7,89,52,2,3,1
3,9397809171349480379,1501601200,1,20170801,-0.411779,-0.353582,-0.266785,1.031804,0.604471,-0.128196,-0.049082,-0.293608,0,5,7,90,13,2,3,1
4,6089902943184578335,1501615525,1,20170801,-0.411779,-0.353582,-0.266785,1.031804,0.604471,-0.128196,-0.049082,-0.293608,0,2,7,90,52,1,3,1


In [52]:
# Loại bỏ các cột định danh không hữu ích cho mô hình
df = df.drop(columns=["fullVisitorId", "visitId"])

# Chuyển cột date sang dạng datetime
df["date"] = pd.to_datetime(df["date"], format="%Y%m%d")

# Tạo thêm các đặc trưng thời gian
df["day_of_week"] = df["date"].dt.dayofweek   # thứ trong tuần (0=thứ2)
df["month"] = df["date"].dt.month
df["is_weekend"] = df["day_of_week"].isin([5,6]).astype(int)

# Xử lý giá trị thiếu (nếu có)
df = df.fillna(df.median(numeric_only=True))
df = df.drop(columns=["date"])

df.head()

,visitNumber,pageviews,hits,time_on_site,bounces,newVisits,transactions,transactionRevenue,sessionQualityDim,deviceCategory,browser,operatingSystem,country,city,source,medium,campaign,day_of_week,month,is_weekend
0,1,-0.411779,-0.353582,-0.266785,1.031804,0.604471,-0.128196,-0.049082,-0.293608,0,2,7,30,52,0,4,1,1,8,0
1,2,-0.411779,-0.353582,-0.266785,1.031804,-1.654340,-0.128196,-0.049082,-0.293608,0,2,7,36,32,2,3,1,1,8,0
2,1,-0.411779,-0.353582,-0.266785,1.031804,0.604471,-0.128196,-0.049082,-0.293608,0,2,7,89,52,2,3,1,1,8,0
3,1,-0.411779,-0.353582,-0.266785,1.031804,0.604471,-0.128196,-0.049082,-0.293608,0,5,7,90,13,2,3,1,1,8,0
4,1,-0.411779,-0.353582,-0.266785,1.031804,0.604471,-0.128196,-0.049082,-0.293608,0,2,7,90,52,1,3,1,1,8,0


In [54]:
# xử lý ngoại lai
import numpy as np

# Xác định các cột định lượng (numeric)
numeric_cols = df.select_dtypes(include=["int64", "float64"]).columns

# Áp dụng winsorization cho từng cột
for col in numeric_cols:
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    # Giới hạn giá trị trong khoảng [lower_bound, upper_bound]
    df[col] = np.clip(df[col], lower_bound, upper_bound)

print("✅ Đã xử lý ngoại lai bằng phương pháp winsorization.")


✅ Đã xử lý ngoại lai bằng phương pháp winsorization.


In [56]:
df.head()

,visitNumber,pageviews,hits,time_on_site,bounces,newVisits,transactions,transactionRevenue,sessionQualityDim,deviceCategory,browser,operatingSystem,country,city,source,medium,campaign,day_of_week,month,is_weekend
0,1.0,-0.411779,-0.353582,-0.266785,1.031804,0.604471,-0.128196,-0.049082,-0.293608,0,2,7,30,52,0,4,1,1,8,0
1,2.0,-0.411779,-0.353582,-0.266785,1.031804,-1.654340,-0.128196,-0.049082,-0.293608,0,2,7,36,32,0,4,1,1,8,0
2,1.0,-0.411779,-0.353582,-0.266785,1.031804,0.604471,-0.128196,-0.049082,-0.293608,0,2,7,89,52,0,4,1,1,8,0
3,1.0,-0.411779,-0.353582,-0.266785,1.031804,0.604471,-0.128196,-0.049082,-0.293608,0,2,7,90,13,0,4,1,1,8,0
4,1.0,-0.411779,-0.353582,-0.266785,1.031804,0.604471,-0.128196,-0.049082,-0.293608,0,2,7,90,52,0,4,1,1,8,0


In [58]:
df.to_csv("dataset.csv", index=False)
print("✅ Đã lưu file dataset.csv.")


✅ Đã lưu file dataset.csv.
